### Задача: Анализ логов действий пользователя
1. Входные данные: Таблица с колонкой raw_events (Array of Structs).
   - Пользователь: user_id
   - События: [{timestamp: 100, type: "click"}, {timestamp: 102, type: "buy"}]
2. Задание (SQL):
    - Использовать explode, чтобы развернуть события.
    - Извлечь поле type из структуры.
    - Посчитать количество событий типа "buy" для каждого пользователя.

In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as F

spark = SparkSession.builder.appName("ComplexTypes_Lab").getOrCreate()

data = [(1, [[100, "click"], [102, "buy"]]), (2, [[200, "login"]])]

In [2]:
# --- ЭТАП 1: Описание Схемы (Матрешка) ---
# Читаем снизу вверх (изнутри наружу):

# 1. Описываем одно событие (внутренний объект)
event_struct = StructType([
    StructField("timestamp", LongType(), True),
    StructField("type", StringType(), True)
])

# 2. Описываем всю таблицу
schema = StructType([
    StructField("user_id", IntegerType(), True),
    # Самое важное: Массив, который содержит СТРУКТУРЫ (event_struct)
    StructField("raw_events", ArrayType(event_struct), True)
])

df = spark.createDataFrame(data, schema)
df.createOrReplaceTempView("user_logs")

print("Схема:")
df.printSchema()
print("Данные:")
df.show(truncate=False)

Схема:
root
 |-- user_id: integer (nullable = true)
 |-- raw_events: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- timestamp: long (nullable = true)
 |    |    |-- type: string (nullable = true)

Данные:
+-------+--------------------------+
|user_id|raw_events                |
+-------+--------------------------+
|1      |[{100, click}, {102, buy}]|
|2      |[{200, login}]            |
+-------+--------------------------+



In [3]:
sql_exploded = spark.sql("""
    SELECT 
        user_id, 
        explode(raw_events) as single_event_struct 
    FROM user_logs
""")

sql_exploded.show()

+-------+-------------------+
|user_id|single_event_struct|
+-------+-------------------+
|      1|       {100, click}|
|      1|         {102, buy}|
|      2|       {200, login}|
+-------+-------------------+



In [4]:
# достаем данные:
sql_str_data = spark.sql("""
    SELECT 
        user_id,
        single_event_struct.type as event_type -- "проваливаемся" внутрь структуры
    FROM (
        SELECT user_id, explode(raw_events) as single_event_struct FROM user_logs
    )
""")
sql_str_data.show()

+-------+----------+
|user_id|event_type|
+-------+----------+
|      1|     click|
|      1|       buy|
|      2|     login|
+-------+----------+



In [7]:
final_task = sql_str_data.filter(F.col("event_type") == "buy").groupBy(F.col("user_id")).agg(
    F.count("*").alias("CNT")
)

final_task.show()

+-------+---+
|user_id|CNT|
+-------+---+
|      1|  1|
+-------+---+

